In [1]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = "ConicalClassificationGithub"
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd drive/My\ Drive
%cd $FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive
/content/drive/My Drive/ConicalClassificationGithub


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_files
import codecs as cs
from load_dataset import *

X_train, y_train, X_valid, y_valid, X_test, y_test = load_and_split()

In [3]:
countVec = CountVectorizer(stop_words="english", binary=True, ngram_range=(1, 1), strip_accents='unicode')
countVec.fit(X_train)
vocab = countVec.vocabulary_
X_vec = countVec.transform(X_train)

In [4]:
%load_ext autoreload
%autoreload 2

from ficlearn.feature_extraction.normalExclusion import NETransformer
ne = NETransformer(vocab=vocab)
ne.fit(X_vec)
X_ne = ne.transform(X_vec).toarray()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import *

svm = OneClassSVM(kernel="linear")
svm.fit(X_ne)

y_pred = svm.predict(ne.transform(countVec.transform(X_valid)).toarray())
y_pred[y_pred < 0] = 0
print(f"Validation accuracy: {accuracy_score(y_valid, y_pred)}")

y_pred = svm.predict(ne.transform(countVec.transform(X_test)).toarray())
y_pred[y_pred < 0] = 0
print(f"Testing accuracy: {accuracy_score(y_test, y_pred)}")

Validation accuracy: 0.9998781206199597
Testing accuracy: 0.9998781255712864
